# Environmental Health Hazards and Population Characteristic Modeling
**Author: Jaclyn Dwyer**

**Project Goal**: The goal of this project is to predict percentage of Low Birth Weights based on California census tracts' environmental health hazard factors in order to determine how to allocate resources for low birth weight newborns in CA.

## Overview

In the previous modeling notebook, models were created in order to obtain the best predictions for percentage of LBW based off of environmental health hazards. For this second part of modeling, in order to try and obtain even more accurate predictions, population characteristics are added to the data. The results of these models are compared to the THE BEST MODEL FROM BEFORE. 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
from sklearn.model_selection import train_test_split
import numpy as np
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from itertools import combinations
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline
from sklearn.feature_selection import RFECV
from sklearn.metrics import r2_score

In [2]:
#read in data
df18 = pd.read_csv('data/merged/df18')

#drop Unamed
df18.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)

## Environmental Health Hazard  and Population Characteristic Models

### Final Data Preparation

Some final data preparation is done before running our models including creating dummy variables and dropping columns.

In [3]:
df18.head(2)

,total_population,california_county,sb_535_disadvantaged,ozone,pm2_5,diesel_pm,drinking_water,pesticides,tox_release,traffic,cleanup_sites,groundwater_threats,haz_waste,imp_water_bodies,solid_waste,pollution_burden_score,lbw,education,linguistic_isolation,unemployment,housing_burden,Pop. Char. Score,less_10_yrs,yrs_11_64,greater_65,hispanic,white,african_american,native_american,asian_american,other,prev_ozone,prev_pm2_5,prev_diesel_pm,prev_drinking_water,prev_tox_release,prev_traffic,prev_groundwater_threats,prev_haz_waste,prev_imp_water_bodies,prev_solid_waste,prev_lbw
0,3174,Fresno,Yes,0.065,15.40,48.524,681.20,2.75,18551.957190,909.14,80.5,45.75,0.795,0,21.75,9.85,7.44,53.3,16.2,17.6,26.0,9.55,18.8,73.6,7.6,65.3,4.2,24.6,0.5,3.5,1.8,0.255228,14.746087,44.23,519.882370,96414.458370,1217.535680,55.75,0.520,0,5.0,5.80253
1,6133,San Bernardino,Yes,0.062,13.31,38.556,904.66,1.37,7494.236622,782.26,66.2,36.00,1.250,5,12.00,10.00,7.04,53.3,33.4,12.3,34.1,9.07,19.7,76.1,4.2,91.1,5.8,0.7,0.3,1.4,0.7,0.465401,13.888224,47.08,604.311803,8122.687693,1232.874128,49.00,1.845,5,2.0,6.38952


In [4]:
#create dummy variables
cc_dummies = pd.get_dummies(df18['california_county'], prefix='cc', drop_first=True)
disadvantaged_dummies = pd.get_dummies(df18['sb_535_disadvantaged'], prefix='disadvantaged', drop_first=True)

df18 = pd.concat([df18, cc_dummies, disadvantaged_dummies], axis=1)

In [5]:
df18.rename(columns = {'Pop. Char. Score': 'pop_char_score', 'cc_Amador ': 'cc_amador', 'cc_Butte ' : 'cc_butte', 
                       'cc_Calaveras ': 'cc_calaveras', 'cc_Colusa ': 'cc_colusa', 'cc_Contra Costa': 'cc_contra_costa', 
                       'cc_Del Norte': 'cc_del_norte', 'cc_El Dorado': 'cc_el_dorado', 'cc_Fresno ': 'cc_fresno', 
                       'cc_Glenn ': 'cc_glenn', 'cc_Humboldt ': 'cc_humboldt', 'cc_Imperial ': 'cc_imperial', 'cc_Inyo ': 'cc_inyo', 
                       'cc_Kern ': 'cc_kern', 'cc_Kings ': 'cc_kings', 'cc_Lake ': 'cc_lake', 'cc_Lassen ': 'cc_lassen', 
                       'cc_Los Angeles': 'cc_los_angeles', 'cc_Madera ': 'cc_madera', 'cc_Marin ': 'cc_marin', 
                       'cc_Mariposa ': 'cc_mariposa', 'cc_Mendocino ': 'cc_mendocino', 'cc_Merced ': 'cc_merced', 'cc_Mono ': 'cc_mono', 
                       'cc_Monterey ': 'cc_monterey', 'cc_Napa ': 'cc_napa', 'cc_Nevada ': 'cc_nevada', 'cc_Orange ': 'cc_orange', 
                       'cc_Placer ': 'cc_placer', 'cc_Plumas ': 'cc_plumas', 'cc_Riverside ': 'cc_riverside', 'cc_Sacramento ': 'cc_sacramento', 
                       'cc_San Benito': 'cc_san_benito', 'cc_San Bernardino': 'cc_san_bernardino', 
                       'cc_San Diego': 'cc_san_diego', 'cc_San Francisco': 'cc_san_francisco', 'cc_San Joaquin': 'cc_san_joaquin', 
                       'cc_San Luis Obispo': 'cc_san_luis_obispo', 'cc_San Mateo': 'cc_san_mateo', 
                       'cc_Santa Barbara': 'cc_santa_barbara', 'cc_Santa Clara': 'cc_santa_clara', 'cc_Santa Cruz': 'cc_santa_cruz', 
                       'cc_Shasta ': 'cc_shasta', 'cc_Sierra ': 'cc_sierra', 'cc_Siskiyou ': 'cc_siskiyou', 'cc_Solano ': 'cc_solano', 
                       'cc_Sonoma ': 'cc_sonoma', 'cc_Stanislaus ': 'cc_stanislaus', 'cc_Sutter ': 'cc_sutter', 'cc_Tehama ': 'cc_tehama', 
                       'cc_Trinity ': 'cc_trinity', 'cc_Tulare ': 'cc_tulare', 'cc_Tuolumne ': 'cc_tuolumne', 'cc_Ventura ': 'cc_ventura', 
                       'cc_Yolo ': 'cc_yolo', 'cc_Yuba ': 'cc_yuba'}, inplace = True)

In [6]:
#drop columns
df18.drop(columns = ['california_county', 'sb_535_disadvantaged', 
                     'pop_char_score'], axis = 1, inplace = True)

In [7]:
df18.head(2)

,total_population,ozone,pm2_5,diesel_pm,drinking_water,pesticides,tox_release,traffic,cleanup_sites,groundwater_threats,haz_waste,imp_water_bodies,solid_waste,pollution_burden_score,lbw,education,linguistic_isolation,unemployment,housing_burden,less_10_yrs,yrs_11_64,greater_65,hispanic,white,african_american,native_american,asian_american,other,prev_ozone,prev_pm2_5,prev_diesel_pm,prev_drinking_water,prev_tox_release,prev_traffic,prev_groundwater_threats,prev_haz_waste,prev_imp_water_bodies,prev_solid_waste,prev_lbw,cc_amador,cc_butte,cc_calaveras,cc_colusa,cc_contra_costa,cc_del_norte,cc_el_dorado,cc_fresno,cc_glenn,cc_humboldt,cc_imperial,cc_inyo,cc_kern,cc_kings,cc_lake,cc_lassen,cc_los_angeles,cc_madera,cc_marin,cc_mariposa,cc_mendocino,cc_merced,cc_mono,cc_monterey,cc_napa,cc_nevada,cc_orange,cc_placer,cc_plumas,cc_riverside,cc_sacramento,cc_san_benito,cc_san_bernardino,cc_san_diego,cc_san_francisco,cc_san_joaquin,cc_san_luis_obispo,cc_san_mateo,cc_santa_barbara,cc_santa_clara,cc_santa_cruz,cc_shasta,cc_sierra,cc_siskiyou,cc_solano,cc_sonoma,cc_stanislaus,cc_sutter,cc_tehama,cc_trinity,cc_tulare,cc_tuolumne,cc_ventura,cc_yolo,cc_yuba,disadvantaged_Yes
0,3174,0.065,15.40,48.524,681.20,2.75,18551.957190,909.14,80.5,45.75,0.795,0,21.75,9.85,7.44,53.3,16.2,17.6,26.0,18.8,73.6,7.6,65.3,4.2,24.6,0.5,3.5,1.8,0.255228,14.746087,44.23,519.882370,96414.458370,1217.535680,55.75,0.520,0,5.0,5.80253,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,6133,0.062,13.31,38.556,904.66,1.37,7494.236622,782.26,66.2,36.00,1.250,5,12.00,10.00,7.04,53.3,33.4,12.3,34.1,19.7,76.1,4.2,91.1,5.8,0.7,0.3,1.4,0.7,0.465401,13.888224,47.08,604.311803,8122.687693,1232.874128,49.00,1.845,5,2.0,6.38952,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### Train Test Split

In [8]:
df18_features = df18.drop(columns = 'lbw', axis = 1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(df18_features, 
                                                    df18['lbw'], 
                                                    random_state=20, 
                                                    test_size=0.2)

### Baseline Model

In [10]:
#fit to train data

#instantiate a linear regression object
baseline = LinearRegression()

#fit the linear regression to the data
baseline = baseline.fit(X_train, y_train)

In [11]:
#predict on train and test set
y_train_pred = baseline.predict(X_train)

y_test_pred = baseline.predict(X_test)

In [12]:
#give true value and predictions
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)

mse_test = mean_squared_error(y_test, y_test_pred)
rmse_test = np.sqrt(mse_test)

In [13]:
print('baseline train: ' + str(rmse))
print('baseline test: ' + str(rmse_test))

baseline train: 0.9945933273746232
baseline test: 1.0056552081673176


In [14]:
#give true value and predictions
r2 = r2_score(y_train, y_train_pred)

#give true value and predictions
r2_test = r2_score(y_test, y_test_pred)

In [15]:
r2

0.5930006147197437

In [16]:
r2_test

0.5614434732223458

The results of the train and test set are very similar not eliciting any concerns for overfitting. When compared to the results of the baseline model using only environmental health hazards for predictions, the added population characteristics dropped the rmse by about 0.10.

In [17]:
#create ols model
outcome = 'lbw'
predictors = df18.drop(['lbw'], axis=1)
pred_sum = '+'.join(predictors.columns)
formula = outcome + '~' + pred_sum

model = ols(formula=formula, data=df18).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lbw   R-squared:                       0.590
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     119.2
Date:                Tue, 04 May 2021   Prob (F-statistic):               0.00
Time:                        12:37:50   Log-Likelihood:                -11036.
No. Observations:                7812   AIC:                         2.226e+04
Df Residuals:                    7718   BIC:                         2.291e+04
Df Model:                          93                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.0028      0.005      0.574      0.566      -0.007       0.012
total_population          7.975e-06    6.5e-06      1.227      0.220   -4.76e-06    2.07e-05
ozone                       22.4755      4.545      4.946      0.000      13.567      31.384
pm2_5                        0.0018      0.016      0.111      0.912      -0.030       0.033
diesel_pm                    0.0012      0.002      0.774      0.439      -0.002       0.004
drinking_water           -6.129e-05      0.000     -0.566      0.571      -0.000       0.000
pesticides               -4.528e-06    4.4e-06     -1.029      0.304   -1.32e-05     4.1e-06
tox_release              -2.631e-06   1.53e-06     -1.715      0.086   -5.64e-06    3.77e-07
traffic                   5.124e-05   2.83e-05      1.809      0.070   -4.28e-06       0.000
cleanup_sites                0.0019      0.001      1.931      0.054   -2.82e-05       0.004
groundwater_threats          0.0001      0.001      0.217      0.829      -0.001       0.001
haz_waste                    0.0194      0.017      1.170      0.242      -0.013       0.052
imp_water_bodies             0.0420      0.037      1.135      0.256      -0.031       0.114
solid_waste                  0.0021      0.005      0.379      0.705      -0.009       0.013
pollution_burden_score      -0.0271      0.018     -1.545      0.122      -0.061       0.007
education                   -0.0044      0.002     -1.962      0.050      -0.009   -3.94e-06
linguistic_isolation        -0.0016      0.002     -0.731      0.465      -0.006       0.003
unemployment                 0.0110      0.003      3.524      0.000       0.005       0.017
housing_burden               0.0020      0.002      1.059      0.290      -0.002       0.006
less_10_yrs                  0.0826      0.162      0.509      0.611      -0.235       0.401
yrs_11_64                    0.0979      0.162      0.604      0.546      -0.220       0.416
greater_65                   0.0985      0.162      0.607      0.544      -0.220       0.417
hispanic                    -0.1755      0.162     -1.081      0.280      -0.494       0.143
white                       -0.1884      0.162     -1.161      0.246      -0.507       0.130
african_american            -0.1600      0.162     -0.986      0.324      -0.478       0.158
native_american             -0.1811      0.163     -1.111      0.266      -0.500       0.138
asian_american              -0.1734      0.162     -1.068      0.285      -0.492       0.145
other                       -0.1522      0.162     -0.937      0.349      -0.470       0.166
prev_ozone                  -0.2598      0.159     -1.633      0.102      -0.572       0.052
prev_pm2_5                  -0.0296      0.016     -1.888      0.059      -0.060       0.001
prev_diesel_pm

### Interactions Model

**Create Interactions**

To try a achieve a lower rmse score, all possible interactions are created as well as cross validations. If the interaction improves the score from the baseline model, the interaction is stored in an interactions list. 

In [18]:
regression = LinearRegression()

X = df18.drop('lbw', axis=1)
y = df18['lbw']

crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(regression, X, y, scoring='neg_root_mean_squared_error', cv=crossvalidation))


interactions = []

feat_combinations = combinations(X.columns, 2)

data = X.copy()
for i, (a, b) in enumerate(feat_combinations):
    data['interaction'] = data[a] * data[b]
    score = np.mean(cross_val_score(regression, data, y, scoring='neg_root_mean_squared_error', cv=crossvalidation))
    if score > baseline:
        interactions.append((a, b, round(score,3)))
    
    if i % 50 == 0:
        print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350


In [ ]:
interactions

In [ ]:
len(interactions)

In [ ]:
def create_interaction(i, dataframe, interactions):
    new_column = interactions[i][0] + '_and_' + interactions[i][1]
    dataframe[new_column] = dataframe[interactions[i][0]] * dataframe[interactions[i][1]]

In [ ]:
for i in range(0,len(interactions)):
    create_interaction(i, df18, interactions)

**Train Test Split**

A second train test split is conducted in order to include the interactions.

In [ ]:
df18_features_i = df18.drop(columns = 'lbw', axis = 1)

In [ ]:
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(df18_features_i, 
                                                    df18['lbw'], 
                                                    random_state=20, 
                                                    test_size=0.2)

**Create Interaction Model**

In [ ]:
#fit to train data

#instantiate a linear regression object
interactions = LinearRegression()

#fit the linear regression to the data
interactions = interactions.fit(X_train_i, y_train_i)

In [ ]:
#predict on train and test set
y_train_pred_i = interactions.predict(X_train_i)

y_test_pred_i = interactions.predict(X_test_i)

In [ ]:
#give true value and predictions
mse_i = mean_squared_error(y_train_i, y_train_pred_i)
rmse_i = np.sqrt(mse_i)

#give true value and predictions
mse_test_i = mean_squared_error(y_test_i, y_test_pred_i)
rmse_test_i = np.sqrt(mse_test_i)

In [ ]:
print('baseline train: ' + str(rmse))
print('baseline test: ' + str(rmse_test))
print('interactions train: ' + str(rmse_i))
print('interactions test: ' + str(rmse_test_i))

The same thing that happened after adding all the interactions with the environmental health hazard models happened again here. Adding the interactions resulted in a lowered rmse for the train set. However, the rmse of the test set increased substantially, indicating that the model is overfitted. In order to try and account for overfitting some features are dropped using feature elimination techniques.

### Select K Best Model

Select K best is used in order to try and eliminate some features to account for overfitting and create improved predictions. This model was previously run with a k equal to 300, 220 200, 175, 150, and 100 in order to obtain the best train and test scores. The results are shown in the graph below.

In [ ]:
# f, axes = plt.subplots(1, figsize=(15,5))
# line = sns.lineplot(x= [300, 220, 200, 175, 150, 100],
#                     y=[1.35, 1.37, 1.37, 1.38, 1.39, 1.40])
# line2 = sns.lineplot(x= [300, 220, 200, 175, 150, 100],
#                     y=[1.82, 1.38, 1.38, 1.38, 1.39, 1.40])
# line.axes.set_title("Kbest Train vs Test Results",fontsize=18)
# line.set_xlabel("K Values",fontsize=15)
# line.set_ylabel("Train & Test Scores",fontsize=15)
# #create proxy artist legent
# blue_line = mlines.Line2D([], [], color='blue', label='Train')
# orange_line = mlines.Line2D([], [], color='orange', label='Test')
# line.legend(handles=[blue_line, orange_line]);

The K value of 175 is selected as any values over 175 start to indicate signs of overfitting. While values less than 175 don't show signs of overfitting the train and test rmse scores start to increase slightly compared to 175.

In [ ]:
selector = SelectKBest(f_regression, k=30)

selector.fit(X_train_i, y_train_i)

In [ ]:
selector.get_support();

In [ ]:
selected_k_columns = X_train_i.columns[selector.get_support()]
removed_k_columns = X_train_i.columns[~selector.get_support()]

kbest = LinearRegression()

#fit the linear regression to the data
kbest = kbest.fit(X_train_i[selected_k_columns], y_train_i)

In [ ]:
#predict on train and test set
y_train_pred_k = kbest.predict(X_train_i[selected_k_columns])

y_test_pred_k = kbest.predict(X_test_i[selected_k_columns])

In [ ]:
#give true value and predictions
mse_k = mean_squared_error(y_train_i, y_train_pred_k)
rmse_k = np.sqrt(mse_k)

#give true value and predictions
mse_test_k = mean_squared_error(y_test_i, y_test_pred_k)
rmse_test_k = np.sqrt(mse_test_k)

In [ ]:
print('baseline train: ' + str(rmse))
print('baseline test: ' + str(rmse_test))
print('interactions train: ' + str(rmse_i))
print('interactions test: ' + str(rmse_test_i))
print('kbest train: ' + str(rmse_k))
print('kbest test: ' + str(rmse_test_k))

In [ ]:
60 = tr
80 = tr 1.2915929593297197, ts 1.2915929593297197
125 = tr 1.2817309715250822, ts 1.3054036846824373
150 = tr 1.2786750746699638, ts 1.30426438869419
200 = tr 1.2656040212863735, ts 1.3157643142615463

The results from the kbest model are show a decrease in the rmse score compared to the baseline model and do not show signs of overfitting as seen in the interactions model. 

### Recursive Feature Elimination

One more feature elimination technique is run on the features selected in the kbest model.
A best subset of features is created by a process of eliminating underperforming features of a model one by one.

In [ ]:
rfe = LinearRegression()
# Create recursive feature eliminator that scores features by root mean squared errors
rfe = RFECV(estimator=rfe, step=1, cv=7,  scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1)

# Fit recursive feature eliminator 
rfe.fit(X_train_i[selected__k_columns], y_train_i)

In [ ]:
#create variables for features selected for model and removed
selected_rfe = X_train_i[selected_k_columns].columns[selector.support_]
removed_rfe = X_train_i[selected_k_columns].columns[~selector.support_]

In [ ]:
ols = LinearRegression()
# Create recursive feature eliminator that scores features by mean squared errors
selector = RFECV(estimator=ols, step=1, cv=7,  scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1)

# Fit recursive feature eliminator 
selector.fit(X_train_i[selected_k_columns], y_train)